In [1]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers

In [2]:
df = pd.read_csv('Data/Data_DJIA.csv', index_col = 1)
df['ADJ_PRC'] =  df['PRC']/df['CFACPR']
price = df.pivot(columns = 'PERMNO', values = 'ADJ_PRC')

In [3]:
code_dict = {24643: 'Alcoa',
             59176: 'Amex',
             19561: 'Boeing',
             14541: 'Chev',
             11308: 'Coke',
             11703: 'Du Pont',
             22592: 'MMM',
             18163: 'P&G',
             14322: 'Sears',
             17830: 'U Tech'}

In [4]:
price.rename(columns = code_dict, inplace = True)
price = price[code_dict.values()]

In [5]:
rtn = price.pct_change()[2:]

In [6]:
mean = rtn.mean().values
std = rtn.std().values
corr = rtn.corr().values
cov = rtn.cov().values

In [7]:
rtn.mean()

PERMNO
Alcoa      0.010971
Amex       0.016781
Boeing     0.013849
Chev       0.010020
Coke       0.017898
Du Pont    0.011673
MMM        0.011580
P&G        0.013074
Sears      0.009936
U Tech     0.010003
dtype: float64

In [8]:
rtn.std()

PERMNO
Alcoa      0.086669
Amex       0.084585
Boeing     0.097071
Chev       0.084926
Coke       0.060055
Du Pont    0.068438
MMM        0.057920
P&G        0.056477
Sears      0.080176
U Tech     0.081194
dtype: float64

In [15]:
#Basic Portfolio Settings
t = 50
a = 2/t
N = len(mean)

P = matrix(a*cov)
q = matrix(-mean)
G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
h = matrix(np.array([0.0 for i in range(N)]), tc='d')
A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
b = matrix(np.array([1.0]), tc='d')

sol = solvers.qp(P,q,G,h,A,b)
solvers.options['show_progress'] = False
w = sol['x']
CE_0 = -sol['primal objective']

In [16]:
print(sol['x'])

[ 5.39e-08]
[ 2.37e-07]
[ 2.11e-07]
[ 3.70e-08]
[ 1.00e+00]
[ 7.27e-08]
[ 6.79e-08]
[ 1.38e-07]
[ 3.33e-08]
[ 3.50e-08]



In [17]:
#Parameter Setting
k_list = np.arange(0.05, 0.20, 0.05)
iter_num = 100

In [18]:
#Examine the effect of errors in means
for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N)*k + 1
        mean_new = np.multiply(mean, z)
    
        P = matrix(a*cov)
        q = matrix(-mean_new)
        G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
        h = matrix(np.array([0.0 for i in range(N)]), tc='d')
        A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
        b = matrix(np.array([1.0]), tc='d')

        sol = solvers.qp(P,q,G,h,A,b)
        w_new = sol['x']
        CE_x = np.dot(mean,w_new)[0] - 0.5*a*np.dot(np.dot(w_new.T, cov), w_new)[0][0]
    
        CEL += (1 - CE_x/CE_0)
    CEL /= iter_num
    print(CEL)

0.010465306131283025
0.01920851673522184
0.04524001037698202
0.04101267135011557


In [19]:
#Examine the effect of errors in variances
for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N)*k + 1
        var = np.diag(cov)
        var_new = np.multiply(var, z)
        cov_new = cov + np.diag(var_new - var)
    
        P = matrix(a*cov_new)
        q = matrix(-mean)
        G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
        h = matrix(np.array([0.0 for i in range(N)]), tc='d')
        A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
        b = matrix(np.array([1.0]), tc='d')

        sol = solvers.qp(P,q,G,h,A,b)
        w_new = sol['x']
        CE_x = np.dot(mean,w_new)[0] - 0.5*a*np.dot(np.dot(w_new.T, cov), w_new)[0][0]
    
        CEL += (1 - CE_x/CE_0)
    CEL /= iter_num
    print(CEL)

2.1044330722475025e-11
1.9622068525571024e-10
7.650899358502272e-11
3.6027982153186146e-10


In [20]:
#Examine the effect of errors in covariances
for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N**2)*k + 1
        var = np.diag(cov)
        upper = np.triu(cov)
        cov_new = np.multiply(upper.reshape(-1), z).reshape((N,N))
        cov_new += cov_new.T
        cov_new = cov_new - np.diag(np.diag(cov_new)) + np.diag(var)
    
        P = matrix(a*cov_new)
        q = matrix(-mean)
        G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
        h = matrix(np.array([0.0 for i in range(N)]), tc='d')
        A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
        b = matrix(np.array([1.0]), tc='d')

        sol = solvers.qp(P,q,G,h,A,b)
        w_new = sol['x']
        CE_x = np.dot(mean,w_new)[0] - 0.5*a*np.dot(np.dot(w_new.T, cov), w_new)[0][0]
    
        CEL += (1 - CE_x/CE_0)
    CEL /= iter_num
    print(CEL)

-1.6056839746703134e-10
2.478995564381137e-10
4.4667543996901315e-10
5.154178972510693e-10
